In [3]:
import requests
from bs4 import BeautifulSoup as bs
import re
import csv
import os
import urllib
import time 
import datetime
from csv import writer

In [4]:
# Une fonction pour DL un pdf (ce serait bien d'ajouter une fonction pour diminuer la taille des fichier)
def download_file(download_url, filename):
    response = urllib.request.urlopen(download_url)    
    file = open(filename + ".pdf", 'wb')
    file.write(response.read())
    file.close()

def append_list_as_row(file_name, list_of_elem):
    # Open file in append mode
    with open(file_name, 'a+', newline='') as write_obj:
        # Create a writer object from csv module
        csv_writer = writer(write_obj)
        # Add contents of list as last row in the csv file
        csv_writer.writerow(list_of_elem)

## Create a .csv file containing all sage-specific URL getting you to "Document" page (~2min 50 depending on server business)

In [35]:


file = open('C:/RD/_SCRAPING/output/output.csv', 'w', newline='')
writer = csv.writer(file)
writer.writerow(['url'])

for page in range(0, 14):

    url = 'https://www.gesteau.fr/rechercher/sage?title=&field_comite_target_id=&field_sage_etat_avancement_value=All&field_sage_sous_etat_avancement_value=All&field_sage_sdage2_value=All&gesteau_field_rw_target_id=&gesteau_field_cw_target_id=&gesteau_field_tw_target_id=&gesteau_field_lw_target_id=&gesteau_paragraph_field_gw=&gesteau_paragraph_field_communes=&rech_admin_sage=&field_milieu_target_id=&type_masse_d_eau=&rech_masse_d_eau=&field_sage_enjeux_value=&field_themes_enjeux_target_id=All&field_sage_regles_value=&field_theme_regle_target_id=All&gesteau_field_communes=&page='
    url2 = 'https://www.gesteau.fr'
    req = requests.get(url+str(page))
    soup = bs(req.text, "lxml")

    table = soup.find_all('table')[0]

    for td in table.find_all('td'):
        for anchor in td.find_all('a'):

            href = anchor['href'].strip()

            link = url2+href+'#edit-group-sage-documents'
            print(link)
            writer.writerow([link])
            time.sleep(0.5)

file.close()


https://www.gesteau.fr/sage/adour-amont#edit-group-sage-documents
https://www.gesteau.fr/sage/adour-aval#edit-group-sage-documents
https://www.gesteau.fr/sage/agly#edit-group-sage-documents
https://www.gesteau.fr/sage/agout#edit-group-sage-documents
https://www.gesteau.fr/sage/aisne-vesle-suippe#edit-group-sage-documents
https://www.gesteau.fr/sage/alagnon#edit-group-sage-documents
https://www.gesteau.fr/sage/allan#edit-group-sage-documents
https://www.gesteau.fr/sage/allier-aval#edit-group-sage-documents
https://www.gesteau.fr/sage/arc-provencal#edit-group-sage-documents
https://www.gesteau.fr/sage/ardeche#edit-group-sage-documents
https://www.gesteau.fr/sage/argens#edit-group-sage-documents
https://www.gesteau.fr/sage/argoat-tregor-goelo#edit-group-sage-documents
https://www.gesteau.fr/sage/arguenon-baie-de-la-fresnaye#edit-group-sage-documents
https://www.gesteau.fr/sage/armancon#edit-group-sage-documents
https://www.gesteau.fr/sage/arroux-bourbince#edit-group-sage-documents
https:/

## Loop over URLs to extract docs AND metadata

In [19]:
with open("C:/RD/_SCRAPING/output/output.csv") as infile:
    #metafile = "C:/RD/_SCRAPING/output/metadata.csv"
    reader = csv.DictReader(infile)
    for link in reader:
        req = requests.get(link['url'])

        soup = bs(req.text, "lxml")

        sage_name = soup.title.string.replace(" | Gest'eau", "")
        
        ## ! Let's get those metadata 

        #* First find the corresponding unique ID (SAGE_CODE)
        sage_code_url = link['url'].replace('#edit-group-sage-documents', '#edit-group-reserve' )
        req_code = requests.get(sage_code_url)
        soup_code = bs(req_code.text, 'lxml')
        sage_code = soup_code.find('div', text=re.compile('Code du SAGE')).find_next('div').text

        #* Then find approval date
        sage_date_url = link['url'].replace('#edit-group-sage-documents', '#edit-group-deroulement' )
        req_date = requests.get(sage_date_url)
        soup_date = bs(req_date.text, 'lxml')
        try :
            sage_date = datetime.datetime.strptime(str.strip(soup_date.find('div', text=re.compile('Arrêté d\'approbation du SAGE')).find_next('div').text), '%d/%m/%Y').strftime("%Y%m%d")
        except:
            sage_date = '00000000'
        #* then progression status
        sage_prog_url = link['url'].replace('#edit-group-sage-documents', '#edit-group-info-gene' )
        req_prog = requests.get(sage_prog_url)
        soup_prog = bs(req_prog.text, 'lxml')
        sage_prog = soup_prog.find('div', text=re.compile("Etat d'avancement")).find_next('div').text

        #* then 'structure porteuse'
        sage_struc_url = link['url'].replace('#edit-group-sage-documents', '#edit-group-structures' )
        req_struc = requests.get(sage_struc_url)
        soup_struc = bs(req_struc.text, 'lxml')
        sage_struc = soup_struc.find('div', text=re.compile("Nom de la structure")).find_next('div').text

        #* Add those metadata to metadata.csv
        row = [sage_code +'_SAGE_'+sage_date,sage_code,'1',sage_name,datetime.date.today().strftime('%Y%m%d'),sage_code,'2021','sage','SAGE','null',sage_prog,'Publié','00000000',sage_prog_url+' ',sage_struc]
        append_list_as_row("C:/RD/_SCRAPING/output/metadata.csv", row)
        #! Create corresponding DIR to store documents :

        dir = "C:/RD/_SCRAPING/output/_documents_bis/" + sage_code +'_SAGE_'+sage_date
        folders = ['0_Procedure','1_PAGD','2_Reglement','3_Autres_documents', 'LOGO' ]
        for folder in folders:
            if not os.path.exists(os.path.join(dir+'/'+folder)) :
                os.makedirs(dir+'/'+folder)

        if os.path.exists(dir):
            print('...')
        #! Let's get those documents !
        
        try:
            # * We need to find item list "documents des SAGE"
            for docs in soup.find('h2', text =re.compile('Documents des SAGE')).find_all_next('div',  class_="item-list")  :
                # * Then we iterate over anchors containing "href" class
                for anchor in docs.find_all('a', href=True): 
                    # * We need to isolate headers of each anchors because I'll have to parse them next
                    for anch in zip(anchor.findPrevious('h3'),anchor.findPrevious('h4')):
                        # * Let's parse each header to filter the documents:
                        _header = str(anchor.findPrevious('h3').text)
                        _subheader = str(anchor.findPrevious('h4').text)
                        if _header == 'Arrêtés et délibérations':
                            dir2 = dir+ '/0_Procedure/'
                        elif _subheader == "Plan d'aménagement et de gestion durable (PAGD)":
                            dir2 = dir + '/1_PAGD/'
                        elif _subheader == "Règlement":
                            dir2 = dir+ '/2_Reglement/'
                        else :
                            dir2 = dir+ '/3_Autres_documents/'
                        url_meta = 'https://www.gesteau.fr'+anchor['href']
                        
                        req_pdf = requests.get(url_meta)
                        soup_pdf = bs(req_pdf.text, "lxml")
                        #* Here we DL each .pdf contained in specifics pages. Warning : there can be many. 
                        #! To improve the method : 
                            #* - Use page's name to name the pdf
                            #* - because there can be many files we need to add an auto-incremented index at the end of file name (without which would overwrite the last downloaded)
                            #* - trim filename to remove whitespace
                        for pdf in soup_pdf.find_all('span', class_="file file--mime-application-pdf file--application-pdf"):
                            
                            url_pdf =  pdf.a['href']
                            pdf_name = pdf.text.strip()
                            print('URL_PDF = ' , url_pdf ,'\n', 'PDF_NAME = ', pdf_name)
                            response = urllib.request.urlopen(url_pdf)
                            file = open(dir2+'/' + pdf_name, 'wb')
                            file.write(response.read())
                            file.close()
                            time.sleep(0.5)
                            print('DOWNLOADED')

                        
        except:
            print('NO DOCUMENT AT ALL')
    


...
URL_PDF =  https://www.gesteau.fr/sites/default/files/gesteau/content_files/document/art_mdu_arrete-modif-perimetre_bva_2014.pdf 
 PDF_NAME =  arrete-modif-perimetre_bva_2014.pdf
DOWNLOADED
URL_PDF =  https://www.gesteau.fr/sites/default/files/gesteau/content_files/document/doc_SAGE06005-1235120691.pdf 
 PDF_NAME =  doc_SAGE06005-1235120691.pdf
DOWNLOADED
URL_PDF =  https://www.gesteau.fr/sites/default/files/gesteau/content_files/document/art_renouvellement_cle_160416.pdf 
 PDF_NAME =  art_renouvellement_cle_160416.pdf
DOWNLOADED
URL_PDF =  https://www.gesteau.fr/sites/default/files/gesteau/content_files/document/artp_modification_cle_1502013.pdf 
 PDF_NAME =  artp_modification_cle_1502013.pdf
DOWNLOADED
URL_PDF =  https://www.gesteau.fr/sites/default/files/gesteau/content_files/document/arrete_portant_modification_de_la_cle_2011.pdf 
 PDF_NAME =  arrete_portant_modification_de_la_cle_2011.pdf
DOWNLOADED
URL_PDF =  https://www.gesteau.fr/sites/default/files/gesteau/content_files/do

In [15]:
with open("C:/RD/_SCRAPING/output/output.csv") as infile:
    #metafile = "C:/RD/_SCRAPING/output/metadata.csv"
    reader = csv.DictReader(infile)
    for link in reader:
        req = requests.get(link['url'])

        soup = bs(req.text, "lxml")

        sage_name = soup.title.string.replace(" | Gest'eau", "")
        
        ## ! Let's get those metadata 

        #* First find the corresponding unique ID (SAGE_CODE)
        sage_code_url = link['url'].replace('#edit-group-sage-documents', '#edit-group-reserve' )
        req_code = requests.get(sage_code_url)
        soup_code = bs(req_code.text, 'lxml')
        sage_code = soup_code.find('div', text=re.compile('Code du SAGE')).find_next('div').text

        #* Then find approval date
        sage_date_url = link['url'].replace('#edit-group-sage-documents', '#edit-group-deroulement' )
        req_date = requests.get(sage_date_url)
        soup_date = bs(req_date.text, 'lxml')
        try :
            sage_date = datetime.datetime.strptime(str.strip(soup_date.find('div', text=re.compile('Arrêté d\'approbation du SAGE')).find_next('div').text), '%d/%m/%Y').strftime("%Y%m%d")
        except:
            sage_date = '00000000'
        #* then progression status
        sage_prog_url = link['url'].replace('#edit-group-sage-documents', '#edit-group-info-gene' )
        req_prog = requests.get(sage_prog_url)
        soup_prog = bs(req_prog.text, 'lxml')
        sage_prog = soup_prog.find('div', text=re.compile("Etat d'avancement")).find_next('div').text

        #* then 'structure porteuse'
        sage_struc_url = link['url'].replace('#edit-group-sage-documents', '#edit-group-structures' )
        req_struc = requests.get(sage_struc_url)
        soup_struc = bs(req_struc.text, 'lxml')
        sage_struc = soup_struc.find('div', text=re.compile("Nom de la structure")).find_next('div').text

        #* Add those metadata to metadata.csv
        row = [sage_code +'_SAGE_'+sage_date,sage_code,'1',sage_name,datetime.date.today().strftime('%Y%m%d'),sage_code,'2021','sage','SAGE','null',sage_prog,'Publié','00000000',sage_prog_url+' ',sage_struc]
        append_list_as_row("C:/RD/_SCRAPING/output/metadata.csv", row)
        #! Create corresponding DIR to store documents :

        dir = "C:/RD/_SCRAPING/output/_documents_bis/" + sage_code +'_SAGE_'+sage_date
        folders = ['0_Procedure','1_PAGD','2_Reglement','3_Autres_documents', 'LOGO' ]
        for folder in folders:
            if not os.path.exists(os.path.join(dir+'/'+folder)) :
                os.makedirs(dir+'/'+folder)

        if os.path.exists(dir):
            print('...')
        #! Let's get those documents !
        
        #try:
            # * We need to iterate over item-list tags because there can be 
            for docs in soup.find('h2', text =re.compile('Documents des SAGE')).find_next('div',  class_="item-list")  :
                print(docs.text)
        # except:
        #     print('no docs')

...
 
Arrêtés et délibérations



Arrêtés de périmètre
 Arrêté du 02 juin 2014 modifiant le périmètre du SAGE de la Basse Vallée de l'Aude Arrêté du 17 avril 2001 fixant le périmètre du SAGE Basse vallée de l'Aude

Arrêtés de composition de CLE
 Arrêté du 19 avril 2016 portant renouvellement de la composition de la CLE du SAGE Basse vallée de l'Aude Arrêté du 15 février 2013 portant modification de la composition de la CLE du SAGE Basse Vallée de l'Aude Arrêté du 5 décembre 2011 portant modification de la composition de la CLE du SAGE Basse vallée de l'Aude  Arrêté du 17 mai 2010 portant renouvellement de la Commission Locale de l'Eau du SAGE Basse Vallée de l'Aude Arrété du 6 novembre 2008 portant modification de la CLE du SAGE Basse vallée de l'Aude 

Arrêtés d'approbation
 Arrêté du 23 mai 2017 portant approbation du SAGE de la Basse Vallée de l'Aude Arrêté interpréfectoral du 15 novembre 2007 portant approbation du SAGE Basse vallée de l'Aude
 
 
Documents constitutifs du SAGE



E